In [ ]:
!pip install requests polars numpy pyarrow plotly seaborn scikit-learn

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pandas-2.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached contourpy-1.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached urllib3-2.5.

In [ ]:
!pip install "opensky-api @ git+https://github.com/openskynetwork/opensky-api.git#subdirectory=python"

  Cloning https://github.com/openskynetwork/opensky-api.git to /private/var/folders/vb/08c5mx3j7js297bw3yyg4j100000gn/T/pip-install-f73k3tju/opensky-api_d8e30a77b2944fc3ac26c3bc0460574a
  Running command git clone --filter=blob:none --quiet https://github.com/openskynetwork/opensky-api.git /private/var/folders/vb/08c5mx3j7js297bw3yyg4j100000gn/T/pip-install-f73k3tju/opensky-api_d8e30a77b2944fc3ac26c3bc0460574a
  Resolved https://github.com/openskynetwork/opensky-api.git to commit 85fdcc10f2d24361ea4313f26d8c032fe90eba7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for opensky-api: filename=opensky_api-1.3.0-py2.py3-none-any.whl size=8347 sha256=383a1bdccac336aa2607bb01fd6c24ddf9d2a1ac70a56f6cb5a026cdf1710c0a
  Stored in directory: /private/var/folders/vb/08c5mx3j7js297bw3yyg4j100000gn/T/pip-ephem-wheel-cache-1608crot/wheels/6b/36/16/783c1004ca4277fc9d3bd99be6439c3a4d6aa98bdaa13c74f

In [21]:
!pip install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [9]:
!pip install tqdm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Download Data

In [12]:
# PIREP Turbulence Reports

import os
import requests
from datetime import datetime
from tqdm import tqdm


BASE_URL = "https://mesonet.agron.iastate.edu/cgi-bin/request/gis/pireps.py"

# Where to store the monthly CSVs
OUT_DIR = "aviation-analytics/data/raw/pireps"
os.makedirs(OUT_DIR, exist_ok=True)

# --------- time range settings ----------
START_YEAR = 2020
END_YEAR = 2024   # inclusive
# ---------------------------------------


def month_iter(start_year: int, end_year: int):
    """Yield (year, month) from start_year-01 to end_year-12 inclusive."""
    year, month = start_year, 1
    while (year < end_year) or (year == end_year and month <= 12):
        yield year, month
        if month == 12:
            year += 1
            month = 1
        else:
            month += 1

months = list(month_iter(START_YEAR, END_YEAR))
for (year, month) in tqdm(months, desc="Downloading monthly PIREPs", unit="month"):
    # Start of this month
    sts = datetime(year, month, 1, 0, 0)

    # Start of next month (for ets)
    if month == 12:
        ets = datetime(year + 1, 1, 1, 0, 0)
    else:
        ets = datetime(year, month + 1, 1, 0, 0)

    out_fname = f"pireps_{year}{month:02d}.csv"
    out_path = os.path.join(OUT_DIR, out_fname)

    if os.path.exists(out_path):
        print(f"Skipping {out_fname} (already exists)")
        continue

    params = {
        "fmt": "csv",
        # time range using ISO strings
        "sts": sts.strftime("%Y-%m-%dT%H:%M:%SZ"),
        "ets": ets.strftime("%Y-%m-%dT%H:%M:%SZ"),
    }

    # print(f"Downloading {out_fname}: {params['sts']} → {params['ets']}")

    success = False
    for attempt in range(1, 4):  # retry up to 3 times
        try:
            resp = requests.get(BASE_URL, params=params, timeout=120)

            # If non-empty and 200 — save it
            if resp.status_code == 200 and resp.text.strip():
                with open(out_path, "w", encoding="utf-8") as f:
                    f.write(resp.text)
                success = True
                break
        except Exception as e:
            pass  # handled by retry

    if not success:
        print(f"✖ FAILED for {out_fname}")

Skipping pireps_202001.csv (already exists)
Skipping pireps_202002.csv (already exists)
Skipping pireps_202003.csv (already exists)
Skipping pireps_202004.csv (already exists)
Skipping pireps_202005.csv (already exists)
Skipping pireps_202006.csv (already exists)


In [ ]:
import polars as pl
import os
import glob

RAW_DIR = "aviation-analytics/data/raw/pireps"
PROC_DIR = "aviation-analytics/data/processed"
os.makedirs(PROC_DIR, exist_ok=True)

# 1. Collect all monthly CSVs
files = sorted(glob.glob(os.path.join(RAW_DIR, "pireps_*.csv")))
print(f"Found {len(files)} files")

if not files:
    raise FileNotFoundError(f"No files matching 'pireps_*.csv' in {RAW_DIR}")

bad_files = []
for f in files:
    # print(f"Checking {f}...")
    try:
        _ = pl.read_csv(f,
            separator=",",
            has_header=True,
            infer_schema_length=50000,
            # truncate_ragged_lines=True,
            encoding="utf-8",
            # low_memory=False,
            # rechunk=True,
            use_pyarrow=True,
            quote_char=None,
        )
        # print("  OK")
    except Exception as e:
        print(f"  FAILED ({f}):", e)
        bad_files.append(f)

print(f"Bad files ({len(bad_files)}):", bad_files)

Found 60 files
  FAILED (aviation-analytics/data/raw/pireps/pireps_202101.csv): could not parse `None` as dtype `f64` at column 'LAT' (column number 10)

The current offset in the file is 33547 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `None` to the `null_values` list.

Original error: ```invalid primitive value found during CSV parsing```
  FAILED (aviation-analytics/data/raw/pireps/pireps_202102.csv): could not parse `None` as dtype `f64` at column 'LAT' (column number 10)

The current offset in the file is 10510 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `None` to the `null_values` list.

Original error: ```invalid primitive value found during CSV pa

In [35]:
import os
import glob
import polars as pl

RAW_DIR = "aviation-analytics/data/raw/pireps"
PROC_DIR = "aviation-analytics/data/processed"
os.makedirs(PROC_DIR, exist_ok=True)

# 1. Collect all monthly CSVs
files = sorted(glob.glob(os.path.join(RAW_DIR, "pireps_*.csv")))
print(f"Found {len(files)} files")

if not files:
    raise FileNotFoundError(f"No files matching 'pireps_*.csv' in {RAW_DIR}")
print(files)
# 2. Read + concatenate
df = pl.concat(
    [pl.read_csv(
        f, 
        quote_char=None, 
        separator=",", 
        infer_schema_length=50000, 
        use_pyarrow=True, 
        encoding="utf-8",
    ) for f in files],
    how="vertical_relaxed",
)

print("Columns:", df.columns)

# 3. Use the TURBULENCE column explicitly (since we know the schema)
if "TURBULENCE" not in df.columns:
    raise ValueError("Expected a 'TURBULENCE' column but did not find one.")

turb_col = "TURBULENCE"

# 4. Filter to rows that actually report turbulence
df_turb = df.filter(
    pl.col(turb_col).is_not_null()
    & (pl.col(turb_col).cast(str).str.strip_chars() != "")
    & (pl.col(turb_col).cast(str).str.strip_chars().str.to_uppercase() != "NONE")
)

print("Total rows:", df.height)
print("Turbulence rows:", df_turb.height)

# 5. Save to parquet
out_path = os.path.join(PROC_DIR, "pireps_turbulence_2020_2024.parquet")
df_turb.write_parquet(out_path)
print("Saved:", out_path)

Found 60 files
['aviation-analytics/data/raw/pireps/pireps_202001.csv', 'aviation-analytics/data/raw/pireps/pireps_202002.csv', 'aviation-analytics/data/raw/pireps/pireps_202003.csv', 'aviation-analytics/data/raw/pireps/pireps_202004.csv', 'aviation-analytics/data/raw/pireps/pireps_202005.csv', 'aviation-analytics/data/raw/pireps/pireps_202006.csv', 'aviation-analytics/data/raw/pireps/pireps_202007.csv', 'aviation-analytics/data/raw/pireps/pireps_202008.csv', 'aviation-analytics/data/raw/pireps/pireps_202009.csv', 'aviation-analytics/data/raw/pireps/pireps_202010.csv', 'aviation-analytics/data/raw/pireps/pireps_202011.csv', 'aviation-analytics/data/raw/pireps/pireps_202012.csv', 'aviation-analytics/data/raw/pireps/pireps_202101.csv', 'aviation-analytics/data/raw/pireps/pireps_202102.csv', 'aviation-analytics/data/raw/pireps/pireps_202103.csv', 'aviation-analytics/data/raw/pireps/pireps_202104.csv', 'aviation-analytics/data/raw/pireps/pireps_202105.csv', 'aviation-analytics/data/raw/pir